# ДЗ 1. Логистическая регрессия

In [2]:
import numpy as np
import pandas as pd

##### Читаем датасет

In [3]:
data = pd.read_csv('Shanghai_HMT_2010.csv')
# Проверим данные
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52584 entries, 0 to 52583
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   No             52584 non-null  int64  
 1   year           52584 non-null  int64  
 2   month          52584 non-null  int64  
 3   day            52584 non-null  int64  
 4   hour           52584 non-null  int64  
 5   season         52584 non-null  int64  
 6   PM_Jingan      24700 non-null  float64
 7   PM_US Post     34039 non-null  float64
 8   PM_Xuhui       25189 non-null  float64
 9   DEWP           52571 non-null  float64
 10  HUMI           52571 non-null  float64
 11  PRES           52556 non-null  float64
 12  TEMP           52571 non-null  float64
 13  cbwd           52572 non-null  object 
 14  Iws            52572 non-null  float64
 15  precipitation  48575 non-null  float64
 16  Iprec          48575 non-null  float64
dtypes: float64(10), int64(6), object(1)
memory usage: 

##### Выбрасываем все NaN'ы

In [4]:
data = data.dropna() 
# Проверим данные
# data.info()

##### Остались категориальные признаки - дропнем их тоже

In [5]:
data = data.drop(['cbwd'], axis=1)
# Проверим данные
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21436 entries, 26304 to 52582
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   No             21436 non-null  int64  
 1   year           21436 non-null  int64  
 2   month          21436 non-null  int64  
 3   day            21436 non-null  int64  
 4   hour           21436 non-null  int64  
 5   season         21436 non-null  int64  
 6   PM_Jingan      21436 non-null  float64
 7   PM_US Post     21436 non-null  float64
 8   PM_Xuhui       21436 non-null  float64
 9   DEWP           21436 non-null  float64
 10  HUMI           21436 non-null  float64
 11  PRES           21436 non-null  float64
 12  TEMP           21436 non-null  float64
 13  Iws            21436 non-null  float64
 14  precipitation  21436 non-null  float64
 15  Iprec          21436 non-null  float64
dtypes: float64(10), int64(6)
memory usage: 2.8 MB


##### Нормализация

In [6]:
data = (data - data.mean()) / data.std()

##### Разобъем данные на два класса по давлению - больше медианы (класс 1) и меньше ее (класс 0)

In [7]:
pres_median = data['PRES'].median()
pres_median

-0.00668796382473264

In [8]:
data['PRES'] = (data['PRES'] > pres_median).astype('int64')

In [9]:
# data.to_csv('data.csv')

## Сравним обучение на кастомном классе логистической регрессии и из sklearn

###  Начнем с sklearn

##### Разбиваем данные на тест и трейн

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, roc_auc_score

X = data.drop(['PRES'], axis=1)
y = data['PRES']

# установим random_state, чтобы результаты эксперимента были воспроизводимы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

model = LogisticRegression()

model.fit(X_train, y_train)

predictions_lr = model.predict(X_test)

err = sum(predictions_lr == y_test) / len(y_test)  # accuracy

print("accuracy =", err)

accuracy = 0.9221091320327962


#####  Коэффициенты модели

In [11]:
model.coef_

array([[ 0.27968073, -0.10891684,  1.63240662,  0.30364664,  0.08925376,
         0.75141337, -0.51767906, -0.11853035,  0.49529419, -0.62644594,
        -1.11960903, -4.28958965, -0.17161046, -0.14383498, -0.07903136]])

###  Далее посмотрим на результаты реализованного класса

##### Разбиваем данные на тест и трейн

In [12]:
import torch as T
from lr import MyLogisticRegression

X = data.drop(['PRES'], axis=1)
y = data['PRES']

# установим random_state, чтобы результаты эксперимента были воспроизводимы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


ModuleNotFoundError: No module named 'torch'

In [13]:
import torch
torch.__version__

ModuleNotFoundError: No module named 'torch'


##### Превратим в numpy - после чего в тензоры

In [ ]:
device = 'cpu'

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

X_train = T.tensor(X_train, dtype=T.float32).to(device)
y_train = T.tensor(y_train, dtype=T.long).to(device)
X_test = T.tensor(X_test, dtype=T.float32).to(device)
y_test = T.tensor(y_test, dtype=T.long).to(device)



num_of_features = X_train.size()[1]
# y_train.size()
# len(X_train)

In [ ]:


my_lr = MyLogisticRegression(device, num_of_features)

lrn_rate = 0.0001
indices = np.arange(len(X_train))

w, b = my_lr.fit(
    X_train,
    y_train,
    lrn_rate,
    indices,
    100,
    0)

In [ ]:
predictions_my = my_lr.predict(X_test)

In [ ]:
err = mean_squared_error(predictions_my, y_test)
print("mean_squared_error =", err)